In [1]:
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, CategoricalParameter, ContinuousParameter
import s3fs

import os
import json

In [2]:
sagemaker_session = sagemaker.Session()

bucket = 'animal-reidentification'

role = sagemaker.get_execution_role()

In [3]:
print(bucket, role)
print(os.getcwd())

animal-reidentification arn:aws:iam::974431750608:role/service-role/AmazonSageMaker-ExecutionRole-20210503T160271
/home/ec2-user/SageMaker/animal-reidentification


In [4]:
data_location = f's3://{bucket}/data/'
annotations_location = f's3://{bucket}/annotations/'

print(f'data_location: {data_location}')
print(f'annotations_location: {annotations_location}')

data_location: s3://animal-reidentification/data/
annotations_location: s3://animal-reidentification/annotations/


## Kick off the instance

In [5]:
# Set hyperparameters
model_name = 'im_noAug'
batch_size = 64
num_epochs = 15

In [6]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='denseNet201_v3.py',
                    source_dir='./kevin/',
                    framework_version='1.8.0',
                    role=role,
                    py_version='py3',
                    instance_count=1,
                    instance_type='ml.g4dn.xlarge', # Sagemaker-managed instance
                    hyperparameters={
                        'batch-size': batch_size,
                        'epochs': num_epochs,
                        'name': f'{model_name}_{num_epochs}epoch_batch{batch_size}',
                    })

estimator.fit({
    'data': data_location,
    'annotations': annotations_location
})

NameError: name 'im_noAug' is not defined